# Goal - Create conversation with artifacts
- Artifacts are presented in the system message (along with their history eventually)
- Artifacts are referenced in the conversation by their nickname
- Artifacts can be edited by the AI by rewriting their content (need to use Claude so that I can present this with XML tags)
- Users can modify artifacts directly and the changes are reflected in the conversation
- Demo CRUD
  - User creates artifact by requesting it - user deletes artifact
  - User retrieves artifact by requesting it
- Demo relationship between artifacts
- Demo anchors in artifacts?

## Scenario
- realestate agent is gathering information about a property (might use tools to get more information)
- they talk about the property
- they make a template email - which generates a new artifact
- they edit the email (making references to other artifacts)
- they send it

## Unknown
- [ ] How do I have the assistant generate an artifact?


In [1]:
%load_ext autoreload
%autoreload 2
from conversation import Artifact, Tool, Conversation


In [2]:
def get_listing(address):
    artifact = Artifact(
        identifier="18bacG4a",
        type="application/json", 
        title="742 Maple Street Listing", 
        content="""\
{
    "address": "742 Maple Street",
    "city": "Cedar Rapids",
    "state": "IA",
    "zip": "52402",
    "price": 185000,
    "beds": 3,
    "baths": 2,
    "sqft": 1450,
    "lot_size": 0.25,
    "year_built": 1978,
    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",
    "features": [           
        "Central air",
        "Forced air heating",
        "Hardwood floors",
        "Updated kitchen",
        "Finished basement",
        "Attached garage",
        "Fenced yard"
    ],
}
""")
    return str(artifact)

def get_comparables(address):
    artifact = Artifact(
        identifier="3baf9f83", 
        type="application/json", 
        title="742 Maple Street Comparables", 
        content="""\
[
    {
        "address": "738 Maple Street",
        "city": "Cedar Rapids", 
        "state": "IA",
        "price": 179900,
        "beds": 3,
        "baths": 2,
        "sqft": 1400,
        "year_built": 1975,
        "last_sold": {
            "date": "2020-03-15",
            "price": 165000
        },
        "distance_miles": 0.1,
        "zestimate": 183000
    },
    {
        "address": "755 Oak Drive",
        "city": "Cedar Rapids",
        "state": "IA", 
        "price": 192000,
        "beds": 3,
        "baths": 2.5,
        "sqft": 1500,
        "year_built": 1980,
        "last_sold": {
            "date": "2021-08-01",
            "price": 180000
        },
        "distance_miles": 0.3,
        "zestimate": 195000
    },
    {
        "address": "729 Elm Court",
        "city": "Cedar Rapids",
        "state": "IA",
        "price": 187500,
        "beds": 3,
        "baths": 2,
        "sqft": 1425,
        "year_built": 1977,
        "last_sold": {
            "date": "2020-11-30",
            "price": 175000
        },
        "distance_miles": 0.2,
        "zestimate": 191000
    }
]
""")
    return str(artifact)

def get_email_template():
    artifact = Artifact(
        identifier="98acb34d", 
        type="text/plain", 
        title="Prospective Buyer Listing Email Template", 
        content="""
Dear {buyer_name},

I wanted to bring to your attention an exciting new listing at {address}, {city}, {state} that I believe would be perfect for you.

This beautiful {beds} bedroom, {baths} bathroom home offers {sqft} square feet of living space and was built in {year_built}. It is currently listed at ${price:,}, which represents excellent value for this desirable neighborhood.

Some key features that make this property stand out:
- Spacious layout with {beds} bedrooms
- {sqft} square feet of living space
- Well-maintained home built in {year_built}
- Current Zestimate: ${zestimate:,}

To give you some context about the local market, there are several comparable properties in the immediate vicinity:
- A similar {beds} bed/{baths} bath home just {distance_miles} miles away recently sold for ${last_sold[price]:,}
- Nearby properties range from ${price:,} to ${price:,} in this area
- Most homes in this neighborhood were built in the 1970s-1980s

Would you like to schedule a viewing of this property? I have several time slots available this week and would be happy to show you around.

Best regards,
Your Real Estate Agent
""")
    return str(artifact)

get_listing_schema = {
    "name": "get_listing",
    "description": "Get details about a specific property listing",
    "input_schema": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "The street address to look up"
            }
        },
        "required": ["address"]
    }
}

get_comparables_schema = {
    "name": "get_comparables",
    "description": "Get comparable property listings in the area",
    "input_schema": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "The street address to find comparables for"
            },
        },
        "required": ["address"]
    }
}

# TODO - fill in the comp blanks
get_email_template_schema = {
    "name": "get_email_template",
    "description": "Get an email template for sending property listings to prospective buyers",
    "input_schema": {
        "type": "object",
        "properties": {
            "buyer_name": {
                "type": "string",
                "description": "Name of the prospective buyer"
            },
            "listing": {
                "type": "object",
                "description": "Property listing details to include in the email"
            }
        },
        "required": ["buyer_name", "listing"]
    }
}


tools = [
    Tool(get_listing_schema, get_listing),
    Tool(get_comparables_schema, get_comparables),
    Tool(get_email_template_schema, get_email_template),
]

In [3]:
c = Conversation(tools=tools)

In [4]:
c.say("Find the listing for 742 Maple Street");


[TextBlock(text='Okay, let me look up that listing using the `get_listing` tool:', type='text'), ToolUseBlock(id='toolu_01WW1vPRyTVK4XVMJiV9HLSj', input={'address': '742 Maple Street'}, name='get_listing', type='tool_use')]
Okay, let me look up that listing using the `get_listing` tool:
Here are the key details about the listing at <a href="#18bacG4a">742 Maple Street</a>:

- 3 bedroom, 2 bathroom ranch home
- 1,450 square feet on a 0.25 acre lot
- Built in 1978, with some updates like new kitchen appliances
- Finished basement, attached 2-car garage, fenced backyard
- Listed for $185,000 in Cedar Rapids, IA

Let me know if you need any other information about this property listing!


In [5]:
c.messages

[{'role': 'user', 'content': 'Find the listing for 742 Maple Street'},
 {'role': 'assistant',
  'content': [TextBlock(text='Okay, let me look up that listing using the `get_listing` tool:', type='text'),
   ToolUseBlock(id='toolu_01WW1vPRyTVK4XVMJiV9HLSj', input={'address': '742 Maple Street'}, name='get_listing', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01WW1vPRyTVK4XVMJiV9HLSj',
    'content': '<artifact identifier=18bacG4a type=application/json title=742 Maple Street Listing>\n{\n    "address": "742 Maple Street",\n    "city": "Cedar Rapids",\n    "state": "IA",\n    "zip": "52402",\n    "price": 185000,\n    "beds": 3,\n    "baths": 2,\n    "sqft": 1450,\n    "lot_size": 0.25,\n    "year_built": 1978,\n    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",\n

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");